## 🔢 Description Embedding

In [10]:
# Imports
from   dotenv   import load_dotenv
import pandas   as pd
import datetime
import sys
import os

# Add the upper folder to sys.path
import LLMUtils
import Preprocessing

#### Initialization

In [11]:
print("⚡ Start - {} ⚡\n".format(datetime.datetime.now()))
startTime = datetime.datetime.now()

⚡ Start - 2024-07-20 21:51:10.971990 ⚡



#### 📥 1) Load Data 

In [12]:
#DATA_PATH = "../../0_Data/0_AndroCatSet.csv"
#DATA_PATH = "../../0_Data/1_AndroCatSetMini.csv"
#DATA_PATH = "../TmpData/3_MalCatSet_Description.csv"
DATA_PATH = "../TmpData/5_NewMal_Description.csv"

# Read the data
appsDF = pd.read_csv(DATA_PATH, index_col=0)
#appsDF = appsDF.drop(columns=['googlePlayCategoryID'])

# TEST
#appsDF = appsDF.head(2)

print("--- #️⃣ Apps: {} ".format(appsDF.shape[0]))

--- #️⃣ Apps: 942 


In [13]:
appsDF.head(5)

,pkgName,googlePlayDescription
sha256,,
001E53DDA101AF19670FBD7FB5D348B21641036240895F6985BFE2018073E887,es.aroundpixels.hsk3lite,Don't worry: we know that <b>studying Chinese ...
002557D37607615083739ABEB8D6FDC05CD27EC93C11045E61384DA35EC73D92,com.onesports.score,AiScore is your LIVESCORE EXPERT. More informa...
00280B2E4FA921E570467396A380FA4248AFA10372B97EB65668923C1087C854,com.perception.soc.en,"Play the best classic MMORPG, only in Empire O..."
0046DD085DDCA00DD6804614539CC9071BC6A4C541A8FED8CDF364ADFE23FAB6,im.token.app,imToken is a reliable Web3 digital wallet trus...
009AACC5A4276BE4E16C6E5890784E2A230AF16476D27EEB1CE7499FBF0E24DE,com.Siyou.HD,Download this application for free to pilot yo...


#### 🔢 2) Preprocess Description

In [14]:
appsDF['description'] = appsDF['googlePlayDescription'].apply(Preprocessing.preprocessDescription)
appsDF = appsDF.drop(columns=['googlePlayDescription'])
appsDF.head(5)

,pkgName,description
sha256,,
001E53DDA101AF19670FBD7FB5D348B21641036240895F6985BFE2018073E887,es.aroundpixels.hsk3lite,worry know study chinese complicate bingo teac...
002557D37607615083739ABEB8D6FDC05CD27EC93C11045E61384DA35EC73D92,com.onesports.score,expert information statistic completely new ex...
00280B2E4FA921E570467396A380FA4248AFA10372B97EB65668923C1087C854,com.perception.soc.en,play best classic empire online truly massivel...
0046DD085DDCA00DD6804614539CC9071BC6A4C541A8FED8CDF364ADFE23FAB6,im.token.app,reliable web digital wallet trust ten million ...
009AACC5A4276BE4E16C6E5890784E2A230AF16476D27EEB1CE7499FBF0E24DE,com.Siyou.HD,download application free pilot r hd drone use...


#### 🔢 3) Generate Numerical Embeddings

In [15]:
# To interact with ChatGPT
gptManager = LLMUtils.GptManager()

$ Pay here

In [16]:
# Generate Embeddings
appsDF['embedding'] = appsDF['description'].apply(gptManager.generateEmbedding)
appsDF.head(5)

,pkgName,description,embedding
sha256,,,
001E53DDA101AF19670FBD7FB5D348B21641036240895F6985BFE2018073E887,es.aroundpixels.hsk3lite,worry know study chinese complicate bingo teac...,"[0.005222779233008623, -0.01788198947906494, -..."
002557D37607615083739ABEB8D6FDC05CD27EC93C11045E61384DA35EC73D92,com.onesports.score,expert information statistic completely new ex...,"[-0.01127683650702238, 0.0010928395204246044, ..."
00280B2E4FA921E570467396A380FA4248AFA10372B97EB65668923C1087C854,com.perception.soc.en,play best classic empire online truly massivel...,"[0.02538859285414219, 0.04251297935843468, 0.0..."
0046DD085DDCA00DD6804614539CC9071BC6A4C541A8FED8CDF364ADFE23FAB6,im.token.app,reliable web digital wallet trust ten million ...,"[0.012788532301783562, 0.004980458877980709, -..."
009AACC5A4276BE4E16C6E5890784E2A230AF16476D27EEB1CE7499FBF0E24DE,com.Siyou.HD,download application free pilot r hd drone use...,"[0.012929912656545639, 0.030403099954128265, -..."


### 🔢 4) Save into CSV

In [17]:
# AndroCatSet
#appsDF.to_csv("../TmpData/0_AndroCatSetEmbeddings.csv", index=False)

# MalCatSet
#appsDF.to_csv("../TmpData/3_MalCatSet_Embeddings.csv", index=False)

# MalCatSet
appsDF.to_csv("../TmpData/5_NewMal_Embeddings.csv", index=False)

##### 🔚 End

In [18]:
endTime = datetime.datetime.now()
print("\n🔚 --- End - {} --- 🔚".format(endTime))

# Assuming endTime and startTime are in seconds
totalTime = endTime - startTime
minutes = totalTime.total_seconds() // 60
seconds = totalTime.total_seconds() % 60
print("⏱️ --- Time: {:02d} minutes and {:02d} seconds --- ⏱️".format(int(minutes), int(seconds)))


🔚 --- End - 2024-07-20 21:55:58.381243 --- 🔚
⏱️ --- Time: 04 minutes and 47 seconds --- ⏱️
